# Test d'un modèle de NER générique sur des données différentes de LECTAUREP avec spaCy

On essaye d'appliquer un modèle de NER sur des données de nature différente. Pour cela, on utilise un fichier du corpus du projet DAHN (https://github.com/FloChiff/DAHNProject ; https://digitalintellectuals.hypotheses.org/), plus précisément une lettre écrite par Paul d'Estournelles de Constant. Le type de langage, ici correspondant au genre épistolaire, est totalement différent de ce que l'on peut rencontrer dans les pages des répertoires de notaires. De plus, le texte ne contient pas de bruit comme on peut trouver dans les textes résultants de l'HTR.

On sélectionne la lettre 569 : https://github.com/FloChiff/DAHNProject/blob/master/Correspondence/Paul_d_Estournelles_de_Constant/Corpus/Lettre569_3octobre1919.xml

Après avoir récupéré le contenu de la balise `<text>`, un nettoyage à la main léger (suppression des sauts de lignes au sein des paragraphes, suppression des balises `<del>`) a été effectué pour avoir un texte exploitable.

## Imports

In [4]:
import spacy
import srsly

## Fonction d'ouverture du fichier texte

In [7]:
def open_text_file(source):
    with open(source, 'r', encoding='UTF-8') as fh:
        text = fh.read()
    
    return text

## Préannotation NER en vue de la création d'une vérité de terrain

La vérité de terrain NER a été produite à l'aide de [Doccano](https://github.com/doccano/doccano). Le texte utilisé pour constituer une vérité de terrain est identique à celui utilisé pour la prédiction de NER. 

Cette méthodologie de test est plus adapté à l'évaluation d'un modèle entraîné pour une tâche spécifique. Cependant, il a été jugé pertinent de comparer ce qu'il était obtenu avec l'outil de NER brut, et ce qu'on attendrait de cet outil brut **au mieux** : la vérité de terrain. 

Le modèle de langue français utilisé avec spaCy permet de repérer 4 types d'entités : 
* `PER` : personne
* `LOC` : localisation
* `ORG` : organisations
* `MISC` : une entité générique indiquant une entité nommée, sans toutefois l'identifier.

Pour annoter le document, les résultats de NER obtenus avec spaCy ont été importés dans Doccano, puis ont été corrigés manuellement. 

Les corrections de l'annotation automatique concernent : 

* Les entités manquées
* La segmentation des entités
* L'identification des entités
* Suppression des "fausses" entités détectés lors de la prédiction

In [3]:
"""
All credits goes to https://github.com/mchesterkadwell/named-entity-recognition
See https://github.com/mchesterkadwell/named-entity-recognition/blob/main/doccano/doccano_named_ents.py

Modified version of Mary Chester-Kadwell's custom class to generate and output spaCy NER labels in Doccano format from
XML input. (Github https://github.com/mchesterkadwell)

The class has been modified to extract data with a french language model from an text input, in order to quickly
establish a ground truth for NER for benchmarking, but not for training a model.
"""

from pathlib import Path
import json
from bs4 import BeautifulSoup
import fr_core_news_lg

nlp = fr_core_news_lg.load(disable=['tagger', 'parser'])


class DoccanoNamedEnts:
    def __init__(self, xml_folder_path):
        self.xml_path = xml_folder_path
        self.spacy_ents = self._init_spacy_ents()
        self.doccano_ents = self._init_doccano_ents()

    def print(self):
        """Prints all texts and their named entities in Doccano format, encode in UTF8 and decode the string."""
        for item in self.doccano_ents:
            text_first = dict(sorted(item.items(), reverse=True))
            text_first_utf8 = json.dumps(text_first, ensure_ascii=False).encode('utf8')
            print(text_first_utf8.decode())

    def to_file(self, output_file_path):
        """Writes all texts and their named entities in Doccano format to specified file."""
        file = Path(output_file_path).absolute()
        with file.open('w+', encoding='utf-8') as f:
            for item in self.doccano_ents:
                item_ = item.copy()
                item_json = json.dumps(item_, ensure_ascii=False).encode('utf8')
                f.write(item_json.decode())
            print(f'{len(self.doccano_ents)} items saved to {output_file_path}')

    def _init_spacy_ents(self):
        """Returns NER results in spaCy format from the initialised texts."""
        spacy_t = tuple(self.label_spacy_ents(text) for text in self.get_text())
        return spacy_t

    def _init_doccano_ents(self):
        """Returns NER results in Doccano format from the initialised texts."""
        doccano_t = tuple(self.convert_ents(ents) for ents in self.spacy_ents)
        return doccano_t

    def get_text(self):
        """Returns a text extracted from an XMLPage file, and normalize it."""
        with open(self.xml_path) as file:
            text = file.read()
            text = [text]
        return text

    @classmethod
    def convert_ents(cls, spacy_ne):
        """Converts NER results in spaCy format to Doccano format."""
        doccano_d = spacy_ne.copy()
        doccano_d['labels'] = doccano_d.pop('ents')
        new_labels = []
        for item in list(doccano_d['labels']):
            values = list(item.values())
            new_labels.append(values)
        doccano_d['labels'] = new_labels
        return doccano_d

    @classmethod
    def label_spacy_ents(cls, text):
        """Returns NER results predicted by spaCy for a single text."""
        doc = nlp(text)
        doc_dict = doc.to_json()
        return {key: value for (key, value) in doc_dict.items() if key in ['text', 'ents']}


In [4]:
labels = DoccanoNamedEnts("../../corpus_test/dahn/Lettre569_3octobre1919.txt")
labels.print()

labels.to_file('spacy_ner_to_doccano/Lettre569_3octobre1919.jsonl')

{"text": "L'inconvénient de mes voyages est que, loin de satisfaire, ils avivent ma curiosité ; ils renouvellent les\nsources de mon attachement à tout ce qui vit, et mon ambition d'être utile. Pour être utiles, commençons par sentir\net par comprendre; il est trop commode de ne pas comprendre.Je comprends trop bien mon pays. Je viens d'y faire un grand voyage qui, pour un Américain, serait une\npromenade de quelques heures, car je n'ai vu que mon département de la Sarthe, et encore pas tout entier. Grâce à\nl'excellente petite auto que je me suis décidé à acheter à New-York et que chacun ici m'envie, je viens d'en faire\nle tour, intérieurement, avec tout autant de méthode et de préparation que s'il s'était agi de \"couvrir\" le\ncontinent américain comme jadis, en 1911, sous vos auspices et par vos soins. Voyage de trois jours et trois nuits pour\nparcourir 500 Km, mais quel parcours ! Et quel pays ! J'étais seul avec mon chauffeur, silencieux et réservé ; j'ai\npu'savourer, sans\nà 

## Evaluation NER 

 On charge le [modèle de langue française](https://spacy.io/models/fr#fr_core_news_lg) de spaCy.

In [5]:
nlp = spacy.load("fr_core_news_lg")

## Traitement de l'échantillon testé

On ouvre le fichier texte et on le passe dans la chaîne de traitement TAL de spaCy.

In [23]:
sample = open_text_file('../../corpus_test/dahn/Lettre569_3octobre1919.txt')
sample_nlp_process = nlp(sample)

## Traitement de la vérité de terrain

On ouvre le fichier jsonl grâce à la librairie srsly et la méthode read_jsonl().

On récupère la vérité de terrain créée avec Doccano dans un dictionnaire sous le format suivant :

```
{entities: [(start, end, type)]}
```

Ce dictionnaire nous servira à évaluer les performances du modèle générique.

In [27]:
filepath = '../../corpus_test/dahn/ground_truth_ner/Lettre569_3octobre1919_spaCy_doccano.jsonl'
# On utilise le package srsly de spaCy pour transformer un fichier JSONL -format d'export de Doccano- en une liste de dictionnaires
annotations = list(srsly.read_jsonl(filepath))

In [28]:
annotations

[{'id': 14,
  'data': 'L\'inconvénient de mes voyages est que, loin de satisfaire, ils avivent ma curiosité ; ils renouvellent les\nsources de mon attachement à tout ce qui vit, et mon ambition d\'être utile. Pour être utiles, commençons par sentir\net par comprendre; il est trop commode de ne pas comprendre.Je comprends trop bien mon pays. Je viens d\'y faire un grand voyage qui, pour un Américain, serait une\npromenade de quelques heures, car je n\'ai vu que mon département de la Sarthe, et encore pas tout entier. Grâce à\nl\'excellente petite auto que je me suis décidé à acheter à New-York et que chacun ici m\'envie, je viens d\'en faire\nle tour, intérieurement, avec tout autant de méthode et de préparation que s\'il s\'était agi de "couvrir" le\ncontinent américain comme jadis, en 1911, sous vos auspices et par vos soins. Voyage de trois jours et trois nuits pour\nparcourir 500 Km, mais quel parcours ! Et quel pays ! J\'étais seul avec mon chauffeur, silencieux et réservé ; j\'ai\

In [32]:
gt_data = {}
for annotation in annotations :
    gt_entities = {'entities': [(start, end, type_) for start, end, type_ in annotation.get('label')]}
    text_gt = annotation.get('data')

In [33]:
gt_entities

{'entities': [(364, 373, 'LOC'),
  (439, 463, 'LOC'),
  (559, 567, 'LOC'),
  (1516, 1525, 'LOC'),
  (1791, 1797, 'LOC'),
  (2014, 2039, 'MISC'),
  (2112, 2124, 'LOC'),
  (2144, 2153, 'LOC'),
  (2262, 2267, 'LOC'),
  (2770, 2779, 'LOC'),
  (2996, 3001, 'LOC'),
  (3008, 3013, 'LOC'),
  (3135, 3140, 'LOC'),
  (3145, 3150, 'LOC'),
  (3599, 3608, 'LOC'),
  (3701, 3706, 'LOC'),
  (3708, 3714, 'LOC'),
  (3721, 3725, 'LOC'),
  (3763, 3778, 'ORG'),
  (4123, 4127, 'LOC'),
  (4140, 4148, 'LOC'),
  (4160, 4178, 'LOC'),
  (4224, 4238, 'LOC'),
  (4743, 4750, 'LOC'),
  (5669, 5674, 'LOC'),
  (5808, 5816, 'LOC'),
  (6071, 6086, 'LOC'),
  (6390, 6398, 'LOC'),
  (6419, 6426, 'LOC'),
  (6445, 6460, 'LOC'),
  (6763, 6771, 'LOC'),
  (6915, 6922, 'LOC'),
  (7099, 7106, 'LOC'),
  (7532, 7539, 'LOC'),
  (9866, 9881, 'ORG'),
  (9885, 9905, 'LOC'),
  (10122, 10133, 'PER'),
  (10254, 10266, 'ORG'),
  (10351, 10362, 'PER'),
  (10408, 10425, 'ORG'),
  (10437, 10449, 'ORG'),
  (10636, 10642, 'LOC'),
  (10704, 10712

## Vérification de l'alignement entre l'échantillon testé pour la prédiction NER et le texte utilisé pour la vérité de terrain NER

Pour vérifier l'alignement des deux textes utilisés pour la comparaison, on utilise un assert sur les chaînes de caractère de la vérité de terrain et de l'échantillon annotée.

In [34]:
assert text_gt == sample

## Evaluation

On crée un objet Example contenant normalement des informations pour l'entraînement. Il sera utilisé pour stocker le document traité par la pipeline NLP et donc les prédictions de NER, ainsi que la vérité de terrain. 

Voir :
* https://spacy.io/api/example
* https://spacy.io/api/data-formats#dict-input
* https://spacy.io/api/scorer#score_spans

In [35]:
from spacy.training import Example

example = Example.from_dict(sample_nlp_process, gt_entities)

/home/hugoscheithauer/anaconda3/envs/ner_spacy/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "L'inconvénient de mes voyages est que, loin de sat..." with entities "[(364, 373, 'LOC'), (439, 463, 'LOC'), (559, 567, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [37]:
from spacy.scorer import Scorer

scorer = Scorer()
# On récupère les scores.
scores = scorer.score_spans([example], 'ents')

In [38]:
scores

{'ents_p': 0.6341463414634146,
 'ents_r': 0.8013698630136986,
 'ents_f': 0.7080181543116489,
 'ents_per_type': {'ORG': {'p': 0.6923076923076923,
   'r': 0.9,
   'f': 0.7826086956521738},
  'MISC': {'p': 0.046511627906976744, 'r': 1.0, 'f': 0.08888888888888888},
  'LOC': {'p': 0.7803921568627451,
   'r': 0.7991967871485943,
   'f': 0.7896825396825397},
  'PER': {'p': 0.3333333333333333,
   'r': 0.7142857142857143,
   'f': 0.4545454545454545}}}

In [40]:
len_gt_entities = len(gt_entities['entities'])
len_prediction_entities = len(sample_nlp_process.ents)
print(f"nombre d'entités dans la vérité de terrain NER : {len_gt_entities} - nombre d'entités dans la prédiction NER : {len_prediction_entities}")

nombre d'entités dans la vérité de terrain NER : 293 - nombre d'entités dans la prédiction NER : 369


## Analyse

Les scores atteints sur un texte structuré en phrases et propre, dans le sens où on ne retrouve pas le bruit caractéristique généré par l'HTR, sont satisfaisants tout en laissant de la place pour être améliorés. 

Des erreurs caractéristiques, relevées lors de la correction de la prédiction pour la constitution de la vérité de terrain, pourraient certainement être corrigées en affinant un modèle générique sur des données annotées.

Parmis ces erreurs, on retrouve :

* Mauvaise annotation des noms de villes composées. Souvent, le modèle générique annote le nom d'une ville composée comme deux villes différentes. Exemple : `Sceaux LOC -s/ Huisnes PER` 
* Les nationalités sont fréquemment annotées en tant que `LOC`.
* Les hyphénations perturbent l'annotation.
* Lors que le texte change de structure et passe sous le format d'une liste, à la fin de l'échantillon, l'extraction est moins régulière pour les entités `LOC`, là où le modèle générique semble pourtant efficace.

On remarque également que le modèle générique a tendance a sur-annoter, avec 369 entités reconnues, contre 293 dans la vérité de terrain. 

Enfin, la catégorie `MISC` parait toujours aussi difficile à traiter, comment la rendre pertinente ? Ne vaudrait-il pas mieux ne pas le prendre en compte dans les résultats ?

## Visualisation de la prédiction NER

In [41]:
from spacy import displacy

html = displacy.render([sample_nlp_process], style='ent', page=True)